# Lesson 3: Building an Agent Reasoning Loop

## Setup

In [ ]:
from helper import get_openai_api_key
OPENAI_API_KEY = get_openai_api_key()

In [ ]:
import nest_asyncio
nest_asyncio.apply()

## Load the data

To download this paper, below is the needed code:

#!wget "https://openreview.net/pdf?id=VtmBAGCN7o" -O metagpt.pdf

**Note**: The pdf file is included with this lesson. To access it, go to the `File` menu and select`Open...`.

## Setup the Query Tools

In [1]:
from utils import get_doc_tools

vector_tool, summary_tool = get_doc_tools("metagpt.pdf", "metagpt")

## Setup Function Calling Agent

In [2]:
from llama_index.llms.openai import OpenAI

llm = OpenAI(model="gpt-3.5-turbo", temperature=0)

In [3]:
from llama_index.core.agent import FunctionCallingAgentWorker
from llama_index.core.agent import AgentRunner

agent_worker = FunctionCallingAgentWorker.from_tools(
    [vector_tool, summary_tool], 
    llm=llm, 
    verbose=True
)
agent = AgentRunner(agent_worker)

In [4]:
response = agent.query(
    "Tell me about the agent roles in MetaGPT, "
    "and then how they communicate with each other."
)

Added user message to memory: Tell me about the agent roles in MetaGPT, and then how they communicate with each other.
=== Calling Function ===
Calling function: summary_tool_metagpt with args: {"input": "agent roles in MetaGPT"}
=== Function Output ===
Encountered error: Detected nested async. Please use nest_asyncio.apply() to allow nested event loops.Or, use async entry methods like `aquery()`, `aretriever`, `achat`, etc.


/Users/zcui/Workspace/pyenv/envs/rag-embedding/lib/python3.10/site-packages/llama_index/core/tools/calling.py:26: RuntimeWarning: coroutine 'run_async_tasks.<locals>._gather' was never awaited
  return ToolOutput(
/Users/zcui/Workspace/pyenv/envs/rag-embedding/lib/python3.10/site-packages/llama_index/core/tools/calling.py:26: RuntimeWarning: coroutine 'Dispatcher.span.<locals>.async_wrapper' was never awaited
  return ToolOutput(


=== Calling Function ===
Calling function: vector_tool_metagpt with args: {"query": "agent roles in MetaGPT"}
=== Function Output ===
Product Manager, Architect, Project Manager, Engineer, and QA Engineer
=== Calling Function ===
Calling function: summary_tool_metagpt with args: {"input": "how agents communicate with each other in MetaGPT"}
=== Function Output ===
Encountered error: Detected nested async. Please use nest_asyncio.apply() to allow nested event loops.Or, use async entry methods like `aquery()`, `aretriever`, `achat`, etc.
=== Calling Function ===
Calling function: vector_tool_metagpt with args: {"query": "how agents communicate with each other in MetaGPT"}
=== Function Output ===
Agents in MetaGPT communicate with each other by utilizing a shared message pool where they publish structured messages and subscribe to relevant messages based on their profiles. This communication protocol allows agents to exchange information efficiently without the need for direct one-to-one 

In [5]:
print(response.source_nodes[0].get_content(metadata_mode="all"))

page_label: 9
file_name: metagpt.pdf
file_path: metagpt.pdf
file_type: application/pdf
file_size: 16911937
creation_date: 2025-02-09
last_modified_date: 2025-02-09

Preprint
Table 2: Comparison of capabilities for MetaGPT and other approaches. ‘!’ indicates the
presence of a specific feature in the corresponding framework, ‘%’ its absence.
Framework Capabiliy AutoGPT LangChain AgentVerse ChatDev MetaGPT
PRD generation % % % % !
Tenical design genenration % % % % !
API interface generation % % % % !
Code generation ! ! ! ! !
Precompilation execution % % % % !
Role-based task management % % % ! !
Code review % % ! ! !
Table 3: Ablation study on roles. ‘#’ denotes ‘The number of’, ‘Product’ denotes ‘Product man-
ager’, and ‘Project’ denotes ‘Project manager’. ‘ !’ indicates the addition of a specific role. ‘Revi-
sions’ refers to ‘Human Revision Cost’.
Engineer Product Architect Project #Agents #Lines Expense Revisions Executability
! % % % 1 83.0 $ 0.915 10 1.0
! ! % % 2 112.0 $ 1.059 6.

In [6]:
response = agent.chat(
    "Tell me about the evaluation datasets used."
)

Added user message to memory: Tell me about the evaluation datasets used.
=== Calling Function ===
Calling function: summary_tool_metagpt with args: {"input": "evaluation datasets used in MetaGPT"}
=== Function Output ===
Encountered error: Detected nested async. Please use nest_asyncio.apply() to allow nested event loops.Or, use async entry methods like `aquery()`, `aretriever`, `achat`, etc.
=== Calling Function ===
Calling function: vector_tool_metagpt with args: {"query": "evaluation datasets used in MetaGPT"}
=== Function Output ===
The evaluation dataset used in MetaGPT is the SoftwareDev dataset, which includes 70 diverse software development tasks.
=== LLM Response ===
The evaluation dataset used in MetaGPT is the SoftwareDev dataset, which includes 70 diverse software development tasks.


In [7]:
response = agent.chat("Tell me the results over one of the above datasets.")

Added user message to memory: Tell me the results over one of the above datasets.
=== Calling Function ===
Calling function: summary_tool_metagpt with args: {"input": "results over the SoftwareDev dataset in MetaGPT"}
=== Function Output ===
Encountered error: Detected nested async. Please use nest_asyncio.apply() to allow nested event loops.Or, use async entry methods like `aquery()`, `aretriever`, `achat`, etc.
=== Calling Function ===
Calling function: vector_tool_metagpt with args: {"query": "results over the SoftwareDev dataset in MetaGPT"}
=== Function Output ===
MetaGPT achieves an average score of 3.9 over the SoftwareDev dataset, surpassing ChatDev's score of 2.1. Additionally, MetaGPT's structured messaging and feedback mechanisms not only reduce loss of communication information but also improve the execution of code.
=== LLM Response ===
MetaGPT achieves an average score of 3.9 over the SoftwareDev dataset, surpassing ChatDev's score of 2.1. Additionally, MetaGPT's structur

## Lower-Level: Debuggability and Control

In [8]:
agent_worker = FunctionCallingAgentWorker.from_tools(
    [vector_tool, summary_tool], 
    llm=llm, 
    verbose=True
)
agent = AgentRunner(agent_worker)

In [9]:
task = agent.create_task(
    "Tell me about the agent roles in MetaGPT, "
    "and then how they communicate with each other."
)

In [10]:
step_output = agent.run_step(task.task_id)

Added user message to memory: Tell me about the agent roles in MetaGPT, and then how they communicate with each other.
=== Calling Function ===
Calling function: summary_tool_metagpt with args: {"input": "agent roles in MetaGPT"}
=== Function Output ===
Encountered error: Detected nested async. Please use nest_asyncio.apply() to allow nested event loops.Or, use async entry methods like `aquery()`, `aretriever`, `achat`, etc.
=== Calling Function ===
Calling function: summary_tool_metagpt with args: {"input": "how agents communicate with each other in MetaGPT"}
=== Function Output ===
Encountered error: Detected nested async. Please use nest_asyncio.apply() to allow nested event loops.Or, use async entry methods like `aquery()`, `aretriever`, `achat`, etc.


In [11]:
completed_steps = agent.get_completed_steps(task.task_id)
print(f"Num completed for task {task.task_id}: {len(completed_steps)}")
print(completed_steps[0].output.sources[0].raw_output)

Num completed for task 6552b6fb-9341-4d50-868f-bfd77c6c6918: 1
Detected nested async. Please use nest_asyncio.apply() to allow nested event loops.Or, use async entry methods like `aquery()`, `aretriever`, `achat`, etc.


In [12]:
upcoming_steps = agent.get_upcoming_steps(task.task_id)
print(f"Num upcoming steps for task {task.task_id}: {len(upcoming_steps)}")
upcoming_steps[0]

Num upcoming steps for task 6552b6fb-9341-4d50-868f-bfd77c6c6918: 1


TaskStep(task_id='6552b6fb-9341-4d50-868f-bfd77c6c6918', step_id='f71aabe5-9885-465c-ba9e-4830ead48ec6', input=None, step_state={}, next_steps={}, prev_steps={}, is_ready=True)

In [13]:
step_output = agent.run_step(
    task.task_id, input="What about how agents share information?"
)

Added user message to memory: What about how agents share information?
=== Calling Function ===
Calling function: summary_tool_metagpt with args: {"input": "how agents share information in MetaGPT"}
=== Function Output ===
Encountered error: Detected nested async. Please use nest_asyncio.apply() to allow nested event loops.Or, use async entry methods like `aquery()`, `aretriever`, `achat`, etc.


In [16]:
step_output = agent.run_step(task.task_id)
print(step_output.is_last)

=== LLM Response ===
I apologize for the inconvenience. It seems there is an issue with retrieving the information on how agents share information in MetaGPT. Let me try to provide a summary based on my knowledge.

In MetaGPT, agents share information through a collaborative process where each agent contributes to the overall understanding and generation of text. The agents communicate with each other by exchanging information and insights, leveraging their individual capabilities to enhance the collective intelligence of the system. This collaborative approach allows the agents to work together effectively and produce coherent and contextually relevant text outputs.

If you have any specific questions or need further details, please let me know.
True


In [17]:
response = agent.finalize_response(task.task_id)

In [18]:
print(str(response))

I apologize for the inconvenience. It seems there is an issue with retrieving the information on how agents share information in MetaGPT. Let me try to provide a summary based on my knowledge.

In MetaGPT, agents share information through a collaborative process where each agent contributes to the overall understanding and generation of text. The agents communicate with each other by exchanging information and insights, leveraging their individual capabilities to enhance the collective intelligence of the system. This collaborative approach allows the agents to work together effectively and produce coherent and contextually relevant text outputs.

If you have any specific questions or need further details, please let me know.
